In [1]:
import numpy as np
import os
import pickle

from matplotlib import pyplot as plt
%matplotlib inline

from keras.preprocessing.image import load_img,img_to_array
from keras.utils import layer_utils, np_utils
from keras import layers
from keras import models
from keras.callbacks import ModelCheckpoint

from keras.applications import nasnet

In [2]:
IMG_HEIGHT = 256
IMG_WIDTH = 256
RESCALE_POND = 255
PATH_MAIN = os.path.abspath(os.path.join("../.."))
PATH_DRIVE = os.path.join(PATH_MAIN,"data","grapes images")
PATH_TRAIN = os.path.join(PATH_DRIVE, "train")
PATH_TEST = os.path.join(PATH_DRIVE, "validation")

def load_image_from_path(path):
  x = []
  y = []

  images_folders = os.listdir(path)

  for folder in images_folders:
    print(folder)
    images_files_path = os.path.join(path,folder)
    images_file_names = os.listdir(images_files_path)

    for image_file in images_file_names:
      complete_path = os.path.join(images_files_path,image_file)
      images =load_img(complete_path, target_size=(IMG_HEIGHT,IMG_WIDTH))
      x.append(img_to_array(images))
      y.append(folder)

  return x,y

def load_save_image(path_train,path_test):
  x_train, y_train = load_image_from_path(path_train)
  # path = os.path.join(PATH_DRIVE,"x_train_orig.pkl")
  # save_pickle(x_train,path)
  # del x_train
  # path = os.path.join(PATH_DRIVE,"y_train_orig.pkl")
  # save_pickle(y_train,path)
  # del y_train
  x_test,y_test =load_image_from_path(path_test)
  # path = os.path.join(PATH_DRIVE,"x_test_orig.pkl")
  # save_pickle(x_test,path)
  # del x_test
  # path = os.path.join(PATH_DRIVE,"y_test_orig.pkl")
  # save_pickle(y_test,path)
  # del y_test
  return x_train, y_train,x_test,y_test


def encode_by_list(label_list, to_encode_list):
  encode_list =[label_list.index(item) for item in to_encode_list]
  return np_utils.to_categorical(encode_list,len(label_list))

def normalize(np_array):
  return np_array/RESCALE_POND

def save_pickle(object,path):
  pickle.dump(object,open(path,'wb'))

def load_pickle(filename):
  path = os.path.join(PATH_DRIVE,filename)
  result=pickle.load(open(path,'rb'))
  return result

def plot_loss_and_accuracy(model_fitted):
  accuracy = model_fitted.history['acc']
  val_accuracy = model_fitted.history['val_acc']
  loss = model_fitted.history['loss']
  val_loss = model_fitted.history['val_loss']
  epochs = range(len(accuracy))
  plt.plot(epochs, accuracy, 'b', label='Training accuracy')
  plt.plot(epochs, val_accuracy, 'r', label='Validation accuracy')
  plt.ylim(ymin=0)
  plt.ylim(ymax=1)
  plt.xlabel('Epochs ', fontsize=16)
  plt.ylabel('Accuracity', fontsize=16)
  plt.title('Training and validation accuracy', fontsize = 20)
  plt.legend()
  plt.figure()
  plt.plot(epochs, loss, 'b', label='Training loss')
  plt.plot(epochs, val_loss, 'r', label='Validation loss')
  plt.xlabel('Epochs ', fontsize=16)
  plt.ylabel('Loss', fontsize=16)
  plt.title('Training and validation loss', fontsize= 20)
  plt.legend()
  plt.show()

In [3]:
x_train, y_train,x_test,y_test = load_save_image(PATH_TRAIN,PATH_TEST)

Grape___Black_rot
Grape___Esca_(Black_Measles)
Grape___healthy
Grape___Leaf_blight_(Isariopsis_Leaf_Spot)
Grape___Black_rot
Grape___Esca_(Black_Measles)
Grape___healthy
Grape___Leaf_blight_(Isariopsis_Leaf_Spot)


In [4]:
x_train = np.array(x_train)
x_test = np.array(x_test)
labels = list(set(y_train))
y_train = encode_by_list(labels,y_train)
y_train = np.array(y_train)
y_test = encode_by_list(labels,y_test)
y_test = np.array(y_test)

In [5]:
x_train = normalize(x_train)
x_test =  normalize(x_test)

In [6]:
len(x_train), len(y_train),len(x_test), len(y_test)

(7222, 7222, 1805, 1805)

In [7]:
def simple_nn(height,width):
    model = models.Sequential()
    model.add(layers.Flatten(input_shape=(height,width,3),name="Input_layer"))
    model.add(layers.Dense(1000,activation = 'relu', name = "Hidden_layer_1"))
    model.add(layers.Dense(500,activation = 'relu', name = "Hidden_layer_2"))
    model.add(layers.Dense(4,activation = 'softmax', name = "Output_layer"))

    return model

def simple_cnn(height,width):  
  model = models.Sequential()
  
  model.add(layers.Conv2D(32, kernel_size=(3, 3), input_shape=(height, width, 3), activation='relu'))

  model.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
  model.add(layers.MaxPooling2D(pool_size=(2, 2)))

  model.add(layers.Conv2D(128, kernel_size=(3, 3), activation='relu'))
  model.add(layers.Conv2D(256, kernel_size=(3, 3), activation='relu'))

  model.add(layers.Conv2D(512, kernel_size=(3, 3), activation='relu'))
  model.add(layers.MaxPooling2D(pool_size=(2, 2)))
  model.add(layers.Conv2D(1024, kernel_size=(3, 3), activation='relu'))
  model.add(layers.MaxPooling2D(pool_size=(2, 2)))

  model.add(layers.Flatten())
  model.add(layers.Dense(500, activation='relu'))
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(4, activation='softmax'))

  return model

def create_nasnet(height,width):  
  model = nasnet.NASNetMobile(input_shape=(height,width,3), include_top=True, weights=None, input_tensor=None, pooling=None, classes=4)

  return model

In [8]:
EPOCHS = 10
callbacks = [ModelCheckpoint(filepath='weights.{epoch:02d}.h5', monitor='val_acc', save_best_only=True, verbose=1)]

In [ ]:
snn_model = simple_nn(IMG_HEIGHT,IMG_WIDTH)
snn_model.compile(loss = 'categorical_crossentropy', optimizer = 'sgd', metrics = ['acc','mse'])
snn_model.summary()

In [ ]:
snn = snn_model.fit(x=x_train, y = y_train, batch_size = 32, epochs= EPOCHS, verbose=1, validation_data=(x_test,y_test), shuffle= True, callbacks=callbacks)

In [ ]:
cnn_model = simple_cnn(IMG_HEIGHT,IMG_WIDTH)
cnn_model.compile(loss = 'categorical_crossentropy', optimizer = 'sgd', metrics = ['acc','mse'])
cnn_model.summary()

In [ ]:
cnn = cnn_model.fit(x=x_train, y = y_train, batch_size = 32, epochs= EPOCHS, verbose=1, validation_data=(x_test,y_test), shuffle= True,callbacks=callbacks)

https://enmilocalfunciona.io/deep-learning-basico-con-keras-parte-6-nasnet/

In [9]:
nasnet_model = create_nasnet(IMG_HEIGHT,IMG_WIDTH)  
nasnet_model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['acc', 'mse'])
nasnet_model.summary() 

Model: "NASNet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 stem_conv1 (Conv2D)            (None, 127, 127, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 stem_bn1 (BatchNormalization)  (None, 127, 127, 32  128         ['stem_conv1[0][0]']             
                                )                                                            

In [10]:
nasnet = nasnet_model.fit(x=x_train, y=y_train, batch_size=32, epochs=10, verbose=1, validation_data=(x_test, y_test), shuffle=True,  callbacks=callbacks) 

Epoch 1/10
226/226 [==============================] - ETA: 0s - loss: 0.4091 - acc: 0.8520 - mse: 0.0549
Epoch 1: val_acc improved from -inf to 0.23823, saving model to weights.01.h5
226/226 [==============================] - 1608s 7s/step - loss: 0.4091 - acc: 0.8520 - mse: 0.0549 - val_loss: 2.5100 - val_acc: 0.2382 - val_mse: 0.2760
Epoch 2/10
226/226 [==============================] - ETA: 0s - loss: 0.2242 - acc: 0.9140 - mse: 0.0313
Epoch 2: val_acc did not improve from 0.23823
226/226 [==============================] - 1541s 7s/step - loss: 0.2242 - acc: 0.9140 - mse: 0.0313 - val_loss: 1.7872 - val_acc: 0.2382 - val_mse: 0.2362
Epoch 3/10
226/226 [==============================] - ETA: 0s - loss: 0.1569 - acc: 0.9421 - mse: 0.0215
Epoch 3: val_acc did not improve from 0.23823
226/226 [==============================] - 1453s 6s/step - loss: 0.1569 - acc: 0.9421 - mse: 0.0215 - val_loss: 1.4815 - val_acc: 0.2382 - val_mse: 0.2014
Epoch 4/10
226/226 [==============================